In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

import keras
keras.__version__

In [ ]:
# funkcje pomocnicze
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))




# Pierwszy przykład sieci neuronowej

Nie przejmuj się, jeżeli nie zrozumiesz wszystkich elementów tego przykładu. Jest to normalne, jeżeli nie masz doświadczenia w pracy z biblioteką Keras ani innym podobnym do niej pakietem. W związku z tym nie przejmuj się, jeżeli niektóre rzeczy wydadzą Ci się czarną magią! Musimy od czegoś zacząć.

W zaprezentowanym przykładzie próbujemy rozwiązać problem klasyfikacji obrazów w skali szarości przedstawiających ręcznie zapisane cyfry (obrazy te mają rozdzielczość 28x28 pikseli). Chcemy podzielić je na 10 kategorii (cyfry od 0 do 9). Będziemy korzystać ze zbioru danych MNIST, który jest uznawany przez środowisko analityków za zbiór klasyczny. Istnieje on tak długo, jak długa jest historia uczenia maszynowego. Zbiór ten zawiera 60 000 obrazów treningowych oraz 10 000 obrazów testowych. Został on utworzony przez Narodowy Instytut Standaryzacji i Technologii (NIST) w latach 80. ubiegłego wieku. Rozwiązanie wspomnianego problemu można porównać do wyświetlenia napisu „Witaj, świecie!” podczas nauki nowego języka programowania. Zbiór ten jest również używany w celu sprawdzania tego, czy algorytm działa poprawnie. Jeżeli zaczniesz zawodowo zajmować się uczeniem maszynowym, to odkryjesz, że zbiór MNIST pojawia się ciągle w różnych pracach naukowych, artykułach publikowanych w internecie itd. Poniżej pokażemy kilka obrazów z tego zbioru.



```
# Sformatowano jako kod
```

Zbiór danych MNIST jest dołączony do pakietu Keras w formie czterech tablic Numpy:

In [ ]:
#ściągnięcie danych
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Tablice train_images i train_labels tworzą treningowy zbiór danych. Będzie on używany podczas trenowania modelu. Do testowania posłuży nam testowy zbiór danych, składający się z tablic test_images i test_labels. Obrazy są zakodowane w formie tablic Numpy, a etykiety mają formę tablicy cyfr (od 0 do 9). Do każdego obrazu przypisana jest tylko jedna etykieta.

Przyjrzyjmy się treningowemu zbiorowi danych:

In [ ]:
# zobaczmy losowo kilka obrazków ze zbioru testowego
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

idxs = random.sample(range(len(test_images)), len(axes))

for idx, axis in zip(idxs, axes):
    first_image = test_images[idx]
    first_image = np.array(first_image, dtype='float')
    pixels = first_image.reshape((28, 28))
    axis.set_axis_off() 
    axis.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
#sprawdźmy rozmiar treningowej macierzy obrazków


In [ ]:
#sprawdźmy liczbę próbek w zbiorze treningowym


In [ ]:
#zobaczmy jak wyglądają klasy w zbiorze treningowym


In [ ]:
#zobaczmy unikalne klasy w zbiorze treningowym


A teraz zobaczmy, jak wyglądają dane testowe:

In [ ]:
#sprawdźmy rozmiar testowej macierzy obrazków


In [ ]:
#sprawdźmy liczbę próbek w zbiorze testowym


In [ ]:
#zobaczmy unikalne klasy w zbiorze testowym


Będziemy pracować według następującego przepływu roboczego: najpierw będziemy trenować sieć neuronową na danych treningowych: train_images i train_labels. Sieć nauczy się kojarzyć obrazy i etykiety. Następnie nasza sieć wygeneruje przewidywania dotyczące zbioru test_images, a uzyskane wyniki porównamy z etykietami test_labels.

Zbudujmy naszą sieć.

In [ ]:
from keras import models
from keras import layers
from keras.layers import Dropout

#proszę wygenerować sieć o 4 warstwach w kerasie:
# - gęsta z 512 neuronami, funkcją aktywacji ReLU, odpowiednim rozmiarem wejścia
# - dropout ze współczynnikiem odrzucenia 40%
# - gęsta z 256 neuronami, funkcją aktywacji Tanh
# - gęsta z licznością klas predykcji, funkcja aktywacji softmax

network = models.Sequential()


In [ ]:
network.summary()

Głównym blokiem składowym sieci neuronowej jest warstwa (ang. layer). Jest to moduł przetwarzania danych, który można traktować jako filtr danych. Dane wychodzące z filtra mają bardziej przydatną formę od danych do niego wchodzących. Niektóre warstwy dokonują ekstrakcji reprezentacji kierowanych do nich danych — reprezentacje te powinny ułatwiać rozwiązanie problemu, z którym się zmagamy. Większość uczenia głębokiego składa się z łączenia ze sobą prostych warstw w celu zaimplementowania progresywnej destylacji danych. Model uczenia głębokiego jest jak sito przetwarzające dane składające się z coraz drobniejszych siatek — warstw.

Nasza sieć składa się z sekwencji dwóch warstw Dense, które są ze sobą połączone w sposób gęsty (dochodzi tu do gęstego połączenia). Druga warstwa jest dziesięcioelementową warstwą softmax — warstwa ta zwróci tablicę 10 wartości prawdopodobieństwa (suma wszystkich tych wartości jest równa 1). Każdy z tych wyników określa prawdopodobieństwo tego, że na danym obrazie przedstawiono daną cyfrę (obraz może przedstawiać jedną z dziesięciu cyfr).

Na etapie kompilacji musimy określić jeszcze trzy rzeczy w celu przygotowania sieci do trenowania. Są to:

* Funkcja straty — funkcja ta definiuje sposób pomiaru wydajności sieci podczas przetwarzania treningowego zbioru danych, a więc pozwala na dostrajanie parametrów sieci we właściwym kierunku.
* Optymalizator — mechanizm dostrajania sieci na podstawie danych zwracanych przez funkcje straty.
* Metryki monitorowane podczas trenowania i testowania — tutaj interesuje nas jedynie dokładność (część obrazów, która została właściwie sklasyfikowana).

In [ ]:
#proszę skompilować sieć z optimizerem Adam, funkcją kosztu categorical crossentropy i śledzonymi miarami:
#'accuracy', f1_m, precision_m, recall_m



Zanim rozpoczniemy trenowanie, zmienimy kształt danych tak, aby przyjęły kształt oczekiwany przez sieć, i przeskalujemy je do wartości z zakresu [0, 1]. Początkowo nasze obrazy treningowe były zapisywane w postaci macierzy o wymiarach (60000, 28, 28), zawierającej wartości z zakresu [0, 255], i typie uint8. Przekształcamy je w tablicę typu float32 o wymiarach (60000, 28 * 28), zawierającą wartości od 0 do 1.

In [ ]:
# reshape danych treningowych (spłaszczanie obrazków)

# normalizacja danych treningowych poprzez podzielenie przez maksymalną wartość


# reshape danych testowych (spłaszczanie obrazków)

# normalizacja danych testowych poprzez podzielenie przez maksymalną wartość


Musimy dodatkowo zakodować etykiety za pomocą kategorii, czyli zamiast wartości od 0 do 9 chcemy mieć wektory z samymi 0 i jedną 1.



In [ ]:
from tensorflow.keras.utils import to_categorical

# kategoryzacja klas danych treningowych


# kategoryzacja klas danych testowych


In [ ]:
# uczenie utworzonej sieci, liczba epok:10, wielkość batcha: 128
#proszę podać dane testowe jako validation_data w procesie uczenia
#dla chętnych - proszę dodać callback, który stworzy logi dla Tensorboard



In [ ]:
#jeśli utworzono logi za pomocą callback można odkomentować poniższą linijkę

# %load_ext tensorboard

In [ ]:
#jeśli utworzono logi za pomocą callback można odkomentować poniższą linijkę

# tensorboard --logdir=network_logs

Podczas trenowania szybko osiągamy dokładność około 0,98 (98%). Teraz możemy sprawdzić dokładność przetwarzania testowego zbioru danych:


In [ ]:
# inferencja na zbiorze testowym - proszę zapisać wyniki
test_results = None

In [ ]:
#proszę wypisać dokładność zbioru testowego (druga wartość z test_results)



W przypadku testowego zbioru danych również uzyskaliśmy dokładność na poziomie około 98%, zwykle niższą niż na zbiorze treningowym. Różnica między tymi wartościami wynika z nadmiernego dopasowania. Modele uczenia maszynowego mają tendencję do niższej dokładności przetwarzania nowych danych, niż to miało miejsce w przypadku danych treningowych. Zagadnienie to jest głównym tematem rozdziału 3.

To tyle, jeżeli chodzi o nasz pierwszy przykład — właśnie zobaczyłeś, że zbudowanie i wytrenowanie sieci neuronowej klasyfikującej zapis ręczny cyfr może zająć mniej niż 20 linii kodu Pythona.
